In [63]:
import pandas as pd
import json

# Load the JSON data into a DataFrame
with open('result.json') as f:
    json_data = json.load(f)

df = pd.DataFrame(json_data['data'],dtype=str)
df.head()

,siteId,seqNo,date,time,temp1,temp2,temp3,humidity1,humidity2,humidity3,light
0,0,0,2020-10-18,05:30:16,20.8,21.5,21.2,95.8,94,94,1.042
1,0,1,2020-10-18,05:30:33,?,21.5,21.1,?,94,94,1.042
2,0,2,2020-10-18,05:30:48,?,21.4,21.1,?,94,94,1.042
3,0,3,2020-10-18,05:31:03,?,21.4,21.1,?,94,94,1.042
4,0,4,2020-10-18,05:31:18,?,21.5,21.2,?,94,94,1.042


In [64]:
# check for missing values
print(df.isnull().sum())

siteId       0
seqNo        0
date         0
time         0
temp1        0
temp2        0
temp3        0
humidity1    0
humidity2    0
humidity3    0
light        0
dtype: int64


In [65]:
# drop rows with missing values
df.dropna(inplace=True)

In [66]:
# Drop duplicate rows
df=df.drop_duplicates(keep='first')

In [78]:
import numpy as np
# Replace '?' with NaN

df.replace(' ?', np.nan, inplace=True)
# Drop rows containing missing values
# Replace the missing values with NaN
df = df.fillna(np.nan)
# Drop rows containing missing values
# Replace the missing values with NaN

#df = df.replace(['?', 'NA', 'N/A', 'None'],0)

df.tail()


,siteId,seqNo,date,time,temp1,temp2,temp3,humidity1,humidity2,humidity3,light,average_temp,average_humidity
5728,0,5743,2020-10-18,05:26:01,20.8,21.5,21.1,95.7,94,94,1.042,21.133333,94.566667
5735,0,5750,2020-10-18,05:27:46,20.8,21.4,21.1,95.8,94,94,1.042,21.100000,94.600000
5736,0,5751,2020-10-18,05:28:01,20.8,21.5,21.1,95.8,94,94,1.042,21.133333,94.600000
5738,0,5753,2020-10-18,05:28:38,20.8,21.4,21.1,95.5,94,94,1.042,21.100000,94.500000
5740,0,5755,2020-10-18,05:29:01,20.8,21.5,21.1,95.5,94,94,1.042,21.133333,94.500000


In [68]:
# Drop rows containing missing values
df = df.dropna()
df.head()

,siteId,seqNo,date,time,temp1,temp2,temp3,humidity1,humidity2,humidity3,light
0,0,0,2020-10-18,05:30:16,20.8,21.5,21.2,95.8,94,94,1.042
6,0,6,2020-10-18,05:31:46,20.8,21.5,21.1,95.1,94,94,1.042
8,0,8,2020-10-18,05:32:16,20.8,21.4,21.1,95.4,94,94,1.042
9,0,9,2020-10-18,05:32:31,20.8,21.5,21.1,95.3,94,94,1.042
10,0,10,2020-10-18,05:32:46,20.8,21.4,21.1,95.1,94,94,1.042


In [69]:
# Convert temperature columns to numeric
df['temp1'] = pd.to_numeric(df['temp1'], errors='coerce')
df['temp2'] = pd.to_numeric(df['temp2'], errors='coerce')
df['temp3'] = pd.to_numeric(df['temp3'], errors='coerce')

# Convert temperature columns to numeric
df['humidity1'] = pd.to_numeric(df['humidity1'], errors='coerce')
df['humidity2'] = pd.to_numeric(df['humidity2'], errors='coerce')
df['humidity3'] = pd.to_numeric(df['humidity3'], errors='coerce')

df['seqNo'] = pd.to_numeric(df['seqNo'], errors='coerce')

# Calculate the average temperature
df['average_temp'] = df[['temp1', 'temp2', 'temp3']].mean(axis=1)

# Calculate the average humidity
df['average_humidity'] = df[['humidity1', 'humidity2', 'humidity3']].mean(axis=1)

# Create a new DataFrame with only the desired columns
new_df = df[['seqNo','date','time','average_temp', 'average_humidity', 'light']]

# Print the selected columns
print(new_df)

      seqNo         date       time  average_temp  average_humidity   light
0         0   2020-10-18   05:30:16     21.166667         94.600000   1.042
6         6   2020-10-18   05:31:46     21.133333         94.366667   1.042
8         8   2020-10-18   05:32:16     21.100000         94.466667   1.042
9         9   2020-10-18   05:32:31     21.133333         94.433333   1.042
10       10   2020-10-18   05:32:46     21.100000         94.366667   1.042
...     ...          ...        ...           ...               ...     ...
5728   5743   2020-10-18   05:26:01     21.133333         94.566667   1.042
5735   5750   2020-10-18   05:27:46     21.100000         94.600000   1.042
5736   5751   2020-10-18   05:28:01     21.133333         94.600000   1.042
5738   5753   2020-10-18   05:28:38     21.100000         94.500000   1.042
5740   5755   2020-10-18   05:29:01     21.133333         94.500000   1.042

[3736 rows x 6 columns]


In [70]:
new_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3736 entries, 0 to 5740
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   seqNo             3736 non-null   int64  
 1   date              3736 non-null   object 
 2   time              3736 non-null   object 
 3   average_temp      3736 non-null   float64
 4   average_humidity  3736 non-null   float64
 5   light             3736 non-null   object 
dtypes: float64(2), int64(1), object(3)
memory usage: 204.3+ KB


In [71]:
 #Convert columns to string using map(str) method
new_df['time'] = new_df['time'].astype(str)
print(new_df.dtypes)

seqNo                 int64
date                 object
time                 object
average_temp        float64
average_humidity    float64
light                object
dtype: object


/var/folders/x2/zc5k_cm94tqdq5h_t8g_5_rm0000gn/T/ipykernel_87150/2910831007.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['time'] = new_df['time'].astype(str)


In [77]:
new_rows = []  # List to store the combined rows
x=0
count=1
average_temp = 0
average_humidity = 0
light = 0
for index, row in df.iterrows():
    # Access the values of each column in the current row
    time = row['time']

    sliced_text = time[4:6]
  
    
    if(sliced_text=="00"):
      
      if(row['seqNo']-x==1 ):
        
         # Access the values of the current row
        date = row['date']
        time = row['time']
        average_temp = row['average_temp']+average_temp
        average_humidity = row['average_humidity']+average_humidity
        x=row['seqNo']
        count=count+1

        
        



        


        
      else:
        new_row = {
        'date': date,
        'time': time,
        'average_temp': average_temp/count ,
        'average_humidity': average_humidity/count ,
        'light': light }
        
        print(new_row)

        
        date = row['date']
        time = row['time']
        average_temp = row['average_temp']
        average_humidity = row['average_humidity']
        
        count=1

 
       
        x=row['seqNo']
        

        

      

      
      
    

{'date': ' 2020-10-18', 'time': ' 06:00:01', 'average_temp': 0.0, 'average_humidity': 0.0, 'light': 0}
{'date': ' 2020-10-18', 'time': ' 06:00:46', 'average_temp': 21.28333333333333, 'average_humidity': 94.56666666666666, 'light': 0}
{'date': ' 2020-10-18', 'time': ' 07:00:00', 'average_temp': 21.3, 'average_humidity': 94.73333333333333, 'light': 0}
{'date': ' 2020-10-18', 'time': ' 08:00:01', 'average_temp': 22.5, 'average_humidity': 94.39999999999999, 'light': 0}
{'date': ' 2020-10-18', 'time': ' 09:00:00', 'average_temp': 30.125, 'average_humidity': 69.73333333333333, 'light': 0}
{'date': ' 2020-10-18', 'time': ' 09:00:31', 'average_temp': 29.46666666666667, 'average_humidity': 76.16666666666667, 'light': 0}
{'date': ' 2020-10-18', 'time': ' 10:00:00', 'average_temp': 29.36666666666667, 'average_humidity': 77.26666666666667, 'light': 0}
{'date': ' 2020-10-18', 'time': ' 11:00:00', 'average_temp': 36.9, 'average_humidity': 46.28333333333333, 'light': 0}
{'date': ' 2020-10-18', 'time'

from datetime import datetime

start_date = "2022-10-18"
start_datetime = datetime.strptime(start_date, "%Y-%m-%d")
start_unix_time = int(start_datetime.timestamp())

In [ ]:
from datetime import datetime

start_date = "2022-10-18"
start_datetime = datetime.strptime(start_date, "%Y-%m-%d")
start_unix_time = int(start_datetime.timestamp())
print(start_unix_time)

1666031400


In [ ]:
from datetime import datetime

end_date = "2022-10-19"
end_datetime = datetime.strptime(end_date, "%Y-%m-%d")
end_unix_time = int(end_datetime.timestamp())
print(end_unix_time)

1666117800
